In [22]:
import pandas as pd
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define your table path
sentiment_clean = "sentiment-lewagon.sentiment_db.SENTIMENT_TRAIN_ALL"
core = "sentiment-lewagon.sentiment_db.META"

# Load the table into a DataFrame
sc = client.list_rows(sentiment_clean).to_dataframe()
sc.rename(columns={'bucket_filepath': 'bucket_file_path'}, inplace=True)
core = client.list_rows(core).to_dataframe()


sc = sc.drop(columns=['cik','quarter','year','filename','conformed_period_of_report'])

In [23]:
#### CREATE FINAL TABLE
meta = pd.merge(core, sc, on = "bucket_file_path", how = "right")

##### add sector
sector = pd.read_csv('ticker_sector_industry.csv')
sector =sector.drop(columns=['industry'])
##### combine sector and meta
meta_sector = pd.merge(meta, sector, on= 'ticker', how = 'left') ### LJ: 11481 ### RJ 11646

In [28]:
(meta_sector.total_chunks_analysed == 0 ).sum() ## 11635

np.int64(1002)

In [1]:
meta_sector

NameError: name 'meta_sector' is not defined

In [29]:
meta_sector = meta_sector.dropna()
meta_sector.shape

(10633, 29)

In [30]:
meta_sector = meta_sector.drop_duplicates()
meta_sector.shape

(10633, 29)

In [31]:
def upload_fina_table_to_bq_(df):
    """
    Uploads final static table to BQ
    """
    # Rename columns to lowercase
    df.columns = df.columns.str.lower()


    try:
        # Keep only the required columns

        BQ_PROJECT_ID = 'sentiment-lewagon'
        BQ_DATASET_ID = 'sentiment_db'
        BQ_TABLE_ID = 'TRAIN_SET_ALL'
        table_ref = f"{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}"

        client = bigquery.Client()

        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND"
        )

        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()

        print(f"✅ Uploaded {job.output_rows} rows to {table_ref}")

    except Exception as e:
        print(f"❌ Failed to upload DataFrame to BigQuery: {e}")

In [32]:
upload_fina_table_to_bq_(meta_sector)

✅ Uploaded 10633 rows to sentiment-lewagon.sentiment_db.TRAIN_SET_ALL
